# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [ ]:
#url = 'https://jportal.mdcourts.gov/license/pbSearchResult.jsp?slcJurisdiction=50&txtTradeName=Vap%25&txtOwnerName=&txtLocationStreetName=&slcYear=2018&slcSortBy=ownername&pager.offset=0'
#driver = webdriver.Chrome()
#driver.get(url)

#Damn it! It does not work. It redirects to the intro page.

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [ ]:
url = 'https://jportal.mdcourts.gov/license/index_disclaimer.jsp'
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
check_box = driver.find_element_by_id('checkbox')
check_box.click()

In [ ]:
submit_button = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
submit_button.click()

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [ ]:
search_button = driver.find_element_by_xpath('/html/body/table[1]/tbody/tr[2]/td[2]/table/tbody/tr/td[3]/a/img')
search_button.click()

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [ ]:
dropdown_menu = driver.find_element_by_id('slcJurisdiction')
dropdown_menu.send_keys('Statewide')

In [ ]:
dropdown_menu = driver.find_element_by_id('txtTradeName')
dropdown_menu.send_keys('VAP%')

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [ ]:
submit_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]')
submit_button.submit()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [ ]:
#names = driver.find_elements_by_class_name('searchlistitem')
names = driver.find_elements_by_class_name('searchfieldtitle')
#print(names.text)
for name in names:
    print(name.text)

To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [ ]:
detail_button = driver.find_element_by_class_name('searchfieldtitle').find_elements_by_css_selector('td')[2].find_element_by_tag_name('a')
detail_button.click()

Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

In [ ]:
names = driver.find_elements_by_class_name('searchfieldtitle')

for name in names:
    print(name.find_elements_by_css_selector('td')[1].text)
    detail_button = name.find_elements_by_css_selector('td')[2].find_element_by_tag_name('a')
    detail_button.click()
    
    mailing_address = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]')
    print(mailing_address.text)
    location_address = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td')
    print(location_address.text)
    license_information = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]')
    print(license_information.text)
    amount_paid = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[8]/td[2]')
    print(amount_paid.text)
    issued_by = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td')
    print(issued_by.text)
    licenses = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody')
    print(licenses.text)
    
    #return_button = driver.back() # It worked too. Ask if it is a good practice.
    return_button = driver.find_element_by_partial_link_text('Return')
    driver.execute_script("arguments[0].scrollIntoView(true)", return_button)
    return_button.click()
    
    

### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [ ]:
url_list = []

stores = driver.find_elements_by_class_name('searchfieldtitle')

for store in stores:
    
    try:
        
        print(store.find_elements_by_css_selector('td')[1].text)
        name = store.find_elements_by_css_selector('td')[1].text
       
        print(url)
        url = store.find_elements_by_css_selector('td')[2].find_element_by_tag_name('a').get_attribute('href')
  
    except:
        
        print('N/A')
        
    data = {
        'name': name,
        'url': url
        }
    
    url_list.append(data)
      
print(url_list)

df = pd.DataFrame(url_list)
df

### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [ ]:
url_list = []

stores = driver.find_elements_by_class_name('searchfieldtitle')

for store in stores:
    
    try:
        
        print(store.find_elements_by_css_selector('td')[1].text)
        name = store.find_elements_by_css_selector('td')[1].text
       
        print(url)
        url = store.find_elements_by_css_selector('td')[2].find_element_by_tag_name('a').get_attribute('href')
  
    except:
        
        print('N/A')
        
    data = {
        'name': name,
        'url': url
        }
    
    url_list.append(data)

next_button = driver.find_element_by_partial_link_text('Next') # It moves to the next page!
driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
next_button.click()
      
print(url_list)

df = pd.DataFrame(url_list)
df

### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [45]:
# I included all the code in the same cell in order to work faster.

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

url = 'https://jportal.mdcourts.gov/license/pbSearchResult.jsp?slcJurisdiction=50&txtTradeName=Vap%25&txtOwnerName=&txtLocationStreetName=&slcYear=2018&slcSortBy=ownername&pager.offset=0'
driver = webdriver.Chrome()
driver.get(url)

#Damn it!

url = 'https://jportal.mdcourts.gov/license/index_disclaimer.jsp'
driver = webdriver.Chrome()
driver.get(url)

check_box = driver.find_element_by_id('checkbox')
check_box.click()

submit_button = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
submit_button.click()

search_button = driver.find_element_by_xpath('/html/body/table[1]/tbody/tr[2]/td[2]/table/tbody/tr/td[3]/a/img')
search_button.click()

dropdown_menu = driver.find_element_by_id('slcJurisdiction')
dropdown_menu.send_keys('Statewide')

dropdown_menu = driver.find_element_by_id('txtTradeName')
dropdown_menu.send_keys('VAP%')

submit_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]')
submit_button.submit()

url_list = []


scraping = True

while scraping:
    
    results_table = driver.find_element_by_css_selector('table[summary="Search Results Table"]')
    stores = results_table.find_elements_by_css_selector('tr.searchfieldtitle')
    
    print("Scraping a new page")

    for store in stores:

        try:

            #print(store.text)
            #name = store.find_elements_by_css_selector('td')[1].find_element_by_class_name('copybold').text
            name = store.find_element_by_css_selector('td.searchlistitem > span.copybold')
            print(name.text)
            

            #print(url)
            #url = store.find_elements_by_css_selector('td')[2].find_element_by_tag_name('a').get_attribute('href')
            url = store.find_element_by_css_selector('td.searchlistitem + td > a').get_attribute('href')
            print(url)

        except:

            print('N/A')

        #data = {
        #    'name': name,
        #    'url': url
        #    }
        
        #url_list.append(data)

    import time
    time.sleep(5)
    # It has to be out of the for loop
    next_button = driver.find_element_by_partial_link_text('Next') # It moves to the next page!
    driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
    next_button.click()


    
print(url_list)

df = pd.DataFrame(url_list)
df



Scraping a new page
VAPE IT STORE II
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=zj96PpSyHMA%3D
VAPE IT STORE I
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=JgMnB%2FezmZQ%3D
VAPEPAD THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=i5Evytn0Isk%3D
VAPE FROG
N/A
VAPE FROG
N/A
Scraping a new page
VAPE LOFT THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=0WG3XTKayRg%3D
VAPE N CIGAR
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=VNpoep605y0%3D
VAPE DOJO
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=zbRgChs1dug%3D
VAPE HAVEN
N/A
VAPORS LOUNGE
N/A
Scraping a new page
VAPORFI SOUTHSIDE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=5uiCM7%2Ffjfs%3D
VAPE DO JO
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=8LOyaOjyKXY%3D
VAPERTETMD
N/A
VAPERTEKMD
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=4Y5VUP7Xvek%3D
VAPORIOT
N/A
Scraping a new page
VAPE PLUS
https://jpo

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.5 x86_64)


### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [86]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

url = 'https://jportal.mdcourts.gov/license/pbSearchResult.jsp?slcJurisdiction=50&txtTradeName=Vap%25&txtOwnerName=&txtLocationStreetName=&slcYear=2018&slcSortBy=ownername&pager.offset=0'
driver = webdriver.Chrome()
driver.get(url)

url = 'https://jportal.mdcourts.gov/license/index_disclaimer.jsp'
driver = webdriver.Chrome()
driver.get(url)

check_box = driver.find_element_by_id('checkbox')
check_box.click()

submit_button = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
submit_button.click()

search_button = driver.find_element_by_xpath('/html/body/table[1]/tbody/tr[2]/td[2]/table/tbody/tr/td[3]/a/img')
search_button.click()

dropdown_menu = driver.find_element_by_id('slcJurisdiction')
dropdown_menu.send_keys('Statewide')

dropdown_menu = driver.find_element_by_id('txtTradeName')
dropdown_menu.send_keys('VAP%')

submit_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]')
submit_button.submit()

url_list = []

# Avoid the error at the end when there are not more pages
try:

    scraping = True

    while scraping:

        results_table = driver.find_element_by_css_selector('table[summary="Search Results Table"]')
        stores = results_table.find_elements_by_css_selector('tr.searchfieldtitle')

        print("Scraping a new page")

        for store in stores:

            try:

                #print(store.text)
                #name = store.find_elements_by_css_selector('td')[1].find_element_by_class_name('copybold').text
                name = store.find_element_by_css_selector('td.searchlistitem > span.copybold')
                print(name.text)


                #print(url)
                #url = store.find_elements_by_css_selector('td')[2].find_element_by_tag_name('a').get_attribute('href')
                url = store.find_element_by_css_selector('td.searchlistitem + td > a').get_attribute('href')
                print(url)

            except:

                print('N/A')

            data = {
                'name': name.text,
                'url': url
                }

            url_list.append(data)

        import time
        time.sleep(5)
        # It has to be out of the for loop
        next_button = driver.find_element_by_partial_link_text('Next') # It moves to the next page!
        driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
        next_button.click()

# Avoid the error at the end when there are not more pages
except:
    
    scraping = False
    
print(url_list)

df = pd.DataFrame(url_list)
df



Scraping a new page
VAPE IT STORE II
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=tikIgdOCJQ4%3D
VAPE IT STORE I
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=Y6YIE7ANfew%3D
VAPEPAD THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=cMcOrvFiAYQ%3D
VAPE FROG
N/A
VAPE FROG
N/A
Scraping a new page
VAPE LOFT THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=m%2BKX3bl2KY4%3D
VAPE N CIGAR
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=n6qOwkWd1IE%3D
VAPE DOJO
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=GZXZlZgzzuA%3D
VAPE HAVEN
N/A
VAPORS LOUNGE
N/A
Scraping a new page
VAPORFI SOUTHSIDE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ac7QLE5nvkk%3D
VAPE DO JO
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=bemjC13thhk%3D
VAPERTETMD
N/A
VAPERTEKMD
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=kqmsvLvMTDI%3D
VAPORIOT
N/A
Scraping a new page
VAPE PLUS
https://jport

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,https://jportal.mdcourts.gov/license/pbLicense...
4,VAPE FROG,https://jportal.mdcourts.gov/license/pbLicense...
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
8,VAPE HAVEN,https://jportal.mdcourts.gov/license/pbLicense...
9,VAPORS LOUNGE,https://jportal.mdcourts.gov/license/pbLicense...


### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [87]:
df = pd.DataFrame (url_list)
df

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,https://jportal.mdcourts.gov/license/pbLicense...
4,VAPE FROG,https://jportal.mdcourts.gov/license/pbLicense...
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
8,VAPE HAVEN,https://jportal.mdcourts.gov/license/pbLicense...
9,VAPORS LOUNGE,https://jportal.mdcourts.gov/license/pbLicense...


In [51]:
df.to_csv('vape-shops-basic.csv', index=False)

In [ ]:
# Read the csv to check everything is fine

df = pd.read_csv("vape-shops-basic.csv")
df.head()

# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [88]:
df['url'][0]
first_url = df['url'][0]

### Use Selenium to visit that page

In [89]:
driver.get(first_url)

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [90]:
data = {
    'mailing_address': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text,
    'location_address': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text,
    'license_information': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text,
    'amount_paid': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[8]/td[2]').text,
    'issued_by': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text,
    'licenses': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody').text
}
print(data)

{'mailing_address': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804', 'location_address': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801', 'license_information': 'License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury', 'amount_paid': '$ 41.00', 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198', 'licenses': "Code Unit Type of License No. of Licenses Cost\n97 015 OTHER TOBACCO PROD RETLR 1 15.00\n01 002 CHAIN STORE 1 5.00\n71 1.0 TRADER'S LICENSE 1 15.00\nIssuing Fees: 6.00\nTotal: 35.00\nAmount Paid: $ 41.00"}


### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [101]:
urls = df['url']

for url in urls:
    
    driver.get(url)

    scraped_data = {
        'mailing_address': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text,
        'location_address': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text,
        'licence_information': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text,
        'total_amount_paid': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[last()]/td[last()]').text,
        'issued_by': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text
        }
    
    print(scraped_data)

{'mailing_address': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804', 'location_address': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801', 'licence_information': 'License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury', 'total_amount_paid': '$ 41.00', 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}
{'mailing_address': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804', 'location_address': 'VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801', 'licence_information': 'License Status: Issued\nLicense No.: 22375605\nControl No.: 22591855\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury', 'total_amount_paid': '$ 24.00', 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}
{'mailing_address': 'ANJ DISTR

{'mailing_address': 'VAPE JUNGLE LLC\n63 E CHESAPEAKE BEACH ROAD\nOWINGS, MD 20736', 'location_address': 'VAPE JUNGLE\n2070 CRAIN HIGHWAY UNIT F\nWALDORF, MD 20601', 'licence_information': 'License Status: Issued\nLicense No.: 08332318\nControl No.: 08844059\nDate of Issue: 4/09/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 08 Charles County', 'total_amount_paid': '$ 110.00', 'issued_by': 'SHARON L. HANCOCK, CLERK OF CIRCUIT COURT\n200 CHARLES STREET\nLA PLATA, MARYLAND 20646'}
{'mailing_address': 'VAPE JUNGLE LLC\n63 E CHESAPEAKE BEACH ROAD\nOWINGS, MD 20736', 'location_address': 'VAPE JUNGLE\n2070 CRAIN HIGHWAY UNIT F\nWALDORF, MD 20601', 'licence_information': 'License Status: Issued\nLicense No.: 08332318\nControl No.: 08844059\nDate of Issue: 4/09/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 08 Charles County', 'total_amount_paid': '$ 110.00', 'issued_by': 'SHARON L. HANCOCK, CLERK OF CIRCUIT COURT\n200 CHARLES STREET\nLA PLATA, MARYLAND 20646'}
{'mailing_

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [102]:
def scrape_all(row):
    
    try:
        
        driver.get(row['url'])

        scraped_data = {
            'mailing_address': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text,
            'location_address': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text,
            'licence_information': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text,
            'total_amount_paid': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[last()]/td[last()]').text,
            'issued_by': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text
            }

        return pd.Series(scraped_data)
    
    except:
        
        print('Review the code!')

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [103]:
df_after_scraping = df.apply(scrape_all, axis=1)

In [104]:
df_final = df.join(df_after_scraping)
df_final

,name,url,mailing_address,location_address,licence_information,total_amount_paid,issued_by
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, M...",VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALIS...,License Status: Issued\nLicense No.: 22375606\...,$ 41.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. ..."
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, M...",VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\...,License Status: Issued\nLicense No.: 22375605\...,$ 24.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. ..."
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...,ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\...,VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILL...,License Status: Issued\nLicense No.: 02304705\...,$ 94.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8..."
3,VAPE FROG,https://jportal.mdcourts.gov/license/pbLicense...,ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\...,VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILL...,License Status: Issued\nLicense No.: 02304705\...,$ 94.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8..."
4,VAPE FROG,https://jportal.mdcourts.gov/license/pbLicense...,ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\...,VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILL...,License Status: Issued\nLicense No.: 02304705\...,$ 94.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8..."
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...,DISBROW II EMERSON HARRINGTON,VAPE LOFT THE\n185 MITCHELLS CHANCE RD\nEDGEWA...,License Status: Issued\nLicense No.: 02310799\...,$ 154.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8..."
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...,DISCOUNT TOBACCO ESSEX LLC,"VAPE N CIGAR\n7104 MINSTREL UNIT #7\nCOLUMBIA,...",License Status: Issued\nLicense No.: 13343011\...,$ 84.00,"WAYNE A. ROBEY, CLERK OF CIRCUIT COURT\n9250 B..."
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...,FAIRGROUND VILLAGE LLC,VAPE DOJO\n330 ONE FORTY VILLAGE ROAD\nUNIT 15...,License Status: Issued\nLicense No.: 06327188\...,$ 179.00,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n..."
8,VAPE HAVEN,https://jportal.mdcourts.gov/license/pbLicense...,FAIRGROUND VILLAGE LLC,VAPE DOJO\n330 ONE FORTY VILLAGE ROAD\nUNIT 15...,License Status: Issued\nLicense No.: 06327188\...,$ 179.00,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n..."
9,VAPORS LOUNGE,https://jportal.mdcourts.gov/license/pbLicense...,FAIRGROUND VILLAGE LLC,VAPE DOJO\n330 ONE FORTY VILLAGE ROAD\nUNIT 15...,License Status: Issued\nLicense No.: 06327188\...,$ 179.00,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n..."


## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [68]:
df_final.to_csv("vape-total.csv", index=False)

In [ ]:
# Open the csv

df = pd.read_csv("vape-total.csv")
df.head()